# I. US County Race Population

In [12]:
import pandas as pd
import numpy as np 
from datetime import datetime  


def diagnose_Rate(T, **kwargs):
    info_list = []
    for col in T.columns:
        if col in ['Rate_Black', 'Rate_White', 'Rate_Parity', 'FullRate_Black', 'FullRate_White', 'FullRate_Parity']:
            d = {'name': col}
            for k, v in kwargs.items():
                d[k] = v
                
            for k,v in T[col].describe().to_dict().items():
                d[k] = v
            info_list.append(d)

    Report = pd.DataFrame(info_list)
    return Report




path = 'DataMerge/Data/C-AnnualCountyPopulation/US_COUNTY_POPULATION.csv'
US_COUNTY_POPULATION = pd.read_csv(path)
US_COUNTY_POPULATION.head()

,GEOID,State,County,Geographic Area Name,Total_Whole,Total_White,Total_Black,Total_AIndA,Total_Asian,Total_WhiteC,Total_BlackC,Total_AIndAC,Total_AsianC
0,1001,Alabama,Autauga,"Autauga, Alabama",55869,42607,11237,266,656,43563,11784,575,950
1,1003,Alabama,Baldwin,"Baldwin, Alabama",223234,195198,19593,1742,2380,199103,21354,3603,3351
2,1005,Alabama,Barbour,"Barbour, Alabama",24686,12134,11906,170,116,12380,12117,297,152
3,1007,Alabama,Bibb,"Bibb, Alabama",22394,17191,4763,103,48,17420,4907,207,97
4,1009,Alabama,Blount,"Blount, Alabama",57826,55412,978,370,185,56167,1276,820,312


In [13]:
STATE_VACCINE = {}

In [17]:
print("Enter the date of collecting data (Today's data in YYYY-MM-DD):")
Today_Date = input()

assert Today_Date == str(datetime.now().date())

print('Hello, ' + Today_Date)

Enter the date of collecting data (Today's data in YYYY-MM-DD):
2021-05-21
Hello, 2021-05-21


# V. Tableau States

## D. New York (Done)


Solution is from

https://github.com/bertrandmartel/tableau-scraping/issues/9

In [21]:
from tableauscraper import TableauScraper as TS

url = 'https://covid19tracker.health.ny.gov/views/Race_Ethnicity_Public/RacebyCounty'

ts = TS()
ts.loads(url)

workbook = ts.getWorkbook()

parameters = workbook.getParameters()
print(parameters)

ts = TS()
ts.loads(url)

# set parameters column / value
workbook = workbook.setParameter('Show Value as', "Number")
# display worksheets
Data = workbook.getWorksheet('Race').data

[{'column': 'Show Value as', 'values': ['Number', 'Percentage'], 'parameterName': '[Parameters].[Parameter 1]'}]


In [22]:
import pandas as pd

L = []

for county, df in Data[Data['Measure Names-alias'] == 'People with at least one vaccine dose '].groupby('County-alias'):
    county, df
    # print(county)
    if county == '%all%':
        continue
    D = {}
    D['County'] = county
    D['Date'] = Today_Date
    for k, row in df.iterrows():
        d = row.to_dict()
        # print(d)
        Race = d['Race-value'] +'_Num'
        Numb = d['Measure Values-alias']
        Numb = int(Numb.replace(',',''))
        D[Race] = Numb

    L.append(D)

NewYork = pd.DataFrame(L)
NewYork

,County,Date,Other_Num,Asian_Num,African American_Num,White_Num
0,Albany,2021-05-21,1446,12382,13159,119670
1,Allegany,2021-05-21,63,146,90,10279
2,Bronx,2021-05-21,11290,31537,134167,92083
3,Broome,2021-05-21,794,4634,3062,80277
4,Cattaraugus,2021-05-21,594,197,219,21756
...,...,...,...,...,...,...
57,Washington,2021-05-21,133,197,171,25700
58,Wayne,2021-05-21,248,285,807,36086
59,Westchester,2021-05-21,6325,35636,46535,338787
60,Wyoming,2021-05-21,43,93,73,13608


In [23]:
df

,Race-value,Race-alias,Region-value,Region-alias,County-value,County-alias,Measure Names-value,Measure Names-alias,Measure Values-alias
128,Other,Other,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,21
138,Asian,Asian,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,35
148,African American,African American,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,43
158,White,White,Finger Lakes,Finger Lakes,Yates,Yates,[federated.0nvcif80f24s4c147rz6d1mpve6p (copy)...,People with at least one vaccine dose,"8,935"


In [24]:
# print(len(L))

import numpy as np


def convert_integer(v):
    if type(v) == str:
        v = float(int(v.replace(',', '')))
    return v



state = 'New York' ## 

new_L = []

for idx, row in NewYork.iterrows(): ## 
    d = row.to_dict()
    # print(d)
    new_d = {}
    
    if type(d['County']) != str or d['County'] == 'Total' :
        print(d)
        continue 
        
    # try:
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']



    new_d['Numb_White'] = convert_integer(d['White_Num'])
    # new_d['Rate_White'] = convert_percentage(d['White_Rate']) #  if type(d['White_Rate']) == str else d['White_Rate'] 

    new_d['Numb_Black'] = convert_integer(d['African American_Num'])
    # new_d['Rate_Black'] = convert_percentage(d['African American_Rate'])

    new_d['Numb_Asian'] = convert_integer(d['Asian_Num'])
    # new_d['Rate_Asian'] = convert_percentage(d['Asian_Rate'])

    new_d['Date'] = d['Date']

    new_L.append(new_d)

#     except:
#         print(d)

    
Data = pd.DataFrame(new_L)


not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]

print(T.shape)
T.head() # Allogany

(62, 22)


,State,County,Geographic Area Name,Date,Numb_Asian,Numb_Black,Numb_White,Rate_Asian,Rate_AsianC,Rate_Black,...,Rate_WhiteC,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,New York,Albany,"Albany, New York",2021-05-21,12382,13159,119670,0.592866,0.513904,0.305342,...,0.499841,930,2401,20885,24094,43096,48463,231538,239416,305506
1,New York,Allegany,"Allegany, New York",2021-05-21,146,90,10279,0.253033,0.183648,0.135542,...,0.230373,147,324,577,795,664,964,44048,44619,46091
2,New York,Bronx,"Bronx, New York",2021-05-21,31537,134167,92083,0.487540,0.441707,0.216904,...,0.136274,41319,59291,64686,71398,618556,663574,634280,675719,1418207
3,New York,Broome,"Broome, New York",2021-05-21,4634,3062,80277,0.550095,0.455878,0.252724,...,0.475338,519,1680,8424,10165,12116,15644,163473,168884,190488
4,New York,Cattaraugus,"Cattaraugus, New York",2021-05-21,197,219,21756,0.305426,0.210245,0.195013,...,0.304382,2742,3292,645,937,1123,2025,69919,71476,76117


In [25]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Wisconsin', 'New York']


,name,count,mean,std,min,25%,50%,75%,max
0,Rate_Black,62.0,0.224317,0.080467,0.032780,0.167870,0.245780,0.272383,0.486732
1,Rate_White,62.0,0.422387,0.088340,0.145177,0.369996,0.426614,0.487681,0.649718
2,Rate_Parity,62.0,0.198070,0.095794,-0.071726,0.144546,0.194700,0.250330,0.428893


In [ ]:
T.to_csv(state + '.csv')

## D. Wisconsin (Done)

New Method: https://github.com/bertrandmartel/tableau-scraping/issues/5

https://www.dhs.wisconsin.gov/covid-19/vaccine-data.htm#day


In [3]:
# from tableauscraper import TableauScraper as TS

# url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16212677845310/VaccinatedWisconsin-County'

# ts = TS()
# ts.loads(url)

# workbook = ts.getWorkbook()

# # show parameters values / column
# parameters = workbook.getParameters()
# print(parameters)


# workbook = workbook.setParameter("Initiation or Completion", "Residents who have completed the vaccine series")

# worksheet = workbook.getWorksheet("Map")
# dashboard = worksheet.select("County", "Price County")
# df = dashboard.getWorksheet("Race vax/unvax county").data
# df

In [2]:
from tableauscraper import TableauScraper as TS



county = 'Winnebago County'


url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16212677845310/VaccinatedWisconsin-County'

ts = TS()
ts.loads(url)

WorkBook = ts.getWorkbook()

# show parameters values / column
parameters = WorkBook.getParameters()
print(parameters)


# First
# WorkBook = WorkBook.setParameter("Initiation or Completion", 'Residents who have received at least one dose')
# WorkBook = WorkBook.setParameter("Initiation or Completion", "Residents who have completed the vaccine series")
print(type(WorkBook)) # Workbook


WorkSheet_Map = WorkBook.getWorksheet("Map") 
print(type(WorkSheet_Map)) # WorkSheet


SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}
print([i for i in SelectionDict])

WorkBook_County = WorkSheet_Map.select("County", county)
print(type(WorkBook_County)) # Workbook


WorkSheet_County_Race = WorkBook_County.getWorksheet("Race vax/unvax county")
print(type(WorkSheet_County_Race))

df_first = WorkSheet_County_Race.data
df_first

[{'column': 'Initiation or Completion', 'values': ['Residents who have received at least one dose', 'Residents who have completed the vaccine series'], 'parameterName': '[Parameters].[Parameter 3]'}]
<class 'tableauscraper.TableauWorkbook.TableauWorkbook'>
<class 'tableauscraper.TableauWorksheet.TableauWorksheet'>
['County', 'SUM(Number With One Dose)', 'SUM(Number With Two Doses)', 'SUM(Popcount)', 'AGG(Number complete for map)', 'Latitude (generated)', 'Longitude (generated)', 'AGG(% complete)', 'AGG(Calc- Initiation or Full Coverage)', 'AGG(% at least 1 dose)']
<class 'tableauscraper.TableauWorkbook.TableauWorkbook'>
<class 'tableauscraper.TableauWorksheet.TableauWorksheet'>


,Race-value,Race-alias,SUM(Initiation or completed count for TT)-alias,AGG(min(1))-value,AGG(min(1))-alias,ATTR(initiation or completed text for TT)-alias,AGG(Geography TT)-alias,AGG(Calc- Initiation or Full Coverage)-alias,Measure Names-alias
0,1,White,64146,1,1,have received at least one dose,Winnebago County,0.402076,min(1)
1,2,Black,697,1,1,have received at least one dose,Winnebago County,0.132914,min(1)
2,4,Asian,2036,1,1,have received at least one dose,Winnebago County,0.35864,min(1)
3,3,American Indian,365,1,1,have received at least one dose,Winnebago County,0.251898,min(1)


In [3]:
from tableauscraper import TableauScraper as TS

url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16212677845310/VaccinatedWisconsin-County'

ts = TS()
ts.loads(url)

WorkBook = ts.getWorkbook()

# show parameters values / column
parameters = WorkBook.getParameters()
print(parameters)


# Full
# WorkBook = WorkBook.setParameter("Initiation or Completion", 'Residents who have received at least one dose')
WorkBook = WorkBook.setParameter("Initiation or Completion", "Residents who have completed the vaccine series")
print(type(WorkBook)) # Workbook


WorkSheet_Map = WorkBook.getWorksheet("Map") 
print(type(WorkSheet_Map)) # WorkSheet


SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}
print([i for i in SelectionDict])

WorkBook_County = WorkSheet_Map.select("County", county)
print(type(WorkBook_County)) # Workbook


WorkSheet_County_Race = WorkBook_County.getWorksheet("Race vax/unvax county")
print(type(WorkSheet_County_Race))

df_full = WorkSheet_County_Race.data
df_full

[{'column': 'Initiation or Completion', 'values': ['Residents who have received at least one dose', 'Residents who have completed the vaccine series'], 'parameterName': '[Parameters].[Parameter 3]'}]
<class 'tableauscraper.TableauWorkbook.TableauWorkbook'>
<class 'tableauscraper.TableauWorksheet.TableauWorksheet'>
['County', 'SUM(Number With One Dose)', 'SUM(Number With Two Doses)', 'SUM(Popcount)', 'AGG(Number complete for map)', 'Latitude (generated)', 'Longitude (generated)', 'AGG(% complete)', 'AGG(Calc- Initiation or Full Coverage)', 'AGG(% at least 1 dose)']
<class 'tableauscraper.TableauWorkbook.TableauWorkbook'>
<class 'tableauscraper.TableauWorksheet.TableauWorksheet'>


,Race-value,Race-alias,SUM(Initiation or completed count for TT)-alias,AGG(min(1))-value,AGG(min(1))-alias,ATTR(initiation or completed text for TT)-alias,AGG(Geography TT)-alias,AGG(Calc- Initiation or Full Coverage)-alias,Measure Names-alias
0,1,White,57174,1,1,have completed the vaccine series,Winnebago County,0.358375,min(1)
1,2,Black,513,1,1,have completed the vaccine series,Winnebago County,0.097826,min(1)
2,4,Asian,1650,1,1,have completed the vaccine series,Winnebago County,0.290646,min(1)
3,3,American Indian,292,1,1,have completed the vaccine series,Winnebago County,0.201518,min(1)


In [4]:
def process_county(county, df_first, df_full):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df_first.iterrows():
        d = row.to_dict()
        race = d['Race-alias']
        D['Numb_' + race] = d['SUM(Initiation or completed count for TT)-alias']
        # D['Rate_' + race] = d['AGG(Calc- Initiation or Full Coverage)-alias']
        
    for idx, row in df_full.iterrows():
        d = row.to_dict()
        race = d['Race-alias']
        D['FullNumb_' + race] = d['SUM(Initiation or completed count for TT)-alias']
        # D['Rate_' + race] = d['AGG(Calc- Initiation or Full Coverage)-alias']
        
    return D

county = "Price County"

Data_County = process_county(county, df_first, df_full)
# Data_Full = process_county(county, df_full)


Data_County

{'County': 'Price',
 'Numb_White': 64146,
 'Numb_Black': 697,
 'Numb_Asian': 2036,
 'Numb_American Indian': 365,
 'FullNumb_White': 57174,
 'FullNumb_Black': 513,
 'FullNumb_Asian': 1650,
 'FullNumb_American Indian': 292}

In [7]:

from tableauscraper import TableauScraper as TS


def get_Wisconsin_Raw_First():
    
        
    url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16212677845310/VaccinatedWisconsin-County'

    
    #############
    ts = TS()
    ts.loads(url)

    WorkBook = ts.getWorkbook()

    # show parameters values / column
    parameters = WorkBook.getParameters()
    # print(parameters)


    # First
    # WorkBook = WorkBook.setParameter("Initiation or Completion", 'Residents who have received at least one dose')
    # WorkBook = WorkBook.setParameter("Initiation or Completion", "Residents who have completed the vaccine series")
    # print(type(WorkBook)) # Workbook


    WorkSheet_Map = WorkBook.getWorksheet("Map") 
    # print(type(WorkSheet_Map)) # WorkSheet


    SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}
    # print([i for i in SelectionDict])

    # WorkBook_County = WorkSheet_Map.select("County", county)
    # print(type(WorkBook_County)) # Workbook
    # WorkSheet_County_Race = WorkBook_County.getWorksheet("Race vax/unvax county")
    # print(type(WorkSheet_County_Race))
    # df_first = WorkSheet_County_Race.data
    
    
    D_First = {}
    for county in SelectionDict['County']:
        try:
            WorkBook_County = WorkSheet_Map.select("County", county)
            # print(type(WorkBook_County)) # Workbook
            WorkSheet_County_Race = WorkBook_County.getWorksheet("Race vax/unvax county")
            # print(type(WorkSheet_County_Race))

            df_first = WorkSheet_County_Race.data
            D_First[county] = df_first
            print('First', county)
            # df_first
        except:
            print('First -- Fail', county)
    #############
    
    return D_First

    
def get_Wisconsin_Raw_Full():
    url = 'https://bi.wisconsin.gov/t/DHS/views/VaccinesAdministeredtoWIResidents_16212677845310/VaccinatedWisconsin-County'

    #############
    
    ts = TS()
    ts.loads(url)

    WorkBook = ts.getWorkbook()

    # show parameters values / column
    parameters = WorkBook.getParameters()
    # print(parameters)


    # Full
    # WorkBook = WorkBook.setParameter("Initiation or Completion", 'Residents who have received at least one dose')
    WorkBook = WorkBook.setParameter("Initiation or Completion", "Residents who have completed the vaccine series")
    # print(type(WorkBook)) # Workbook


    WorkSheet_Map = WorkBook.getWorksheet("Map") 
    # print(type(WorkSheet_Map)) # WorkSheet


    SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}
    print([i for i in SelectionDict])

    # WorkBook_County = WorkSheet_Map.select("County", county)
    # print(type(WorkBook_County)) # Workbook
    # WorkSheet_County_Race = WorkBook_County.getWorksheet("Race vax/unvax county")
    # print(type(WorkSheet_County_Race))
    # df_full = WorkSheet_County_Race.data
    D_Full = {}
    for county in SelectionDict['County']:

        try:
            WorkBook_County = WorkSheet_Map.select("County", county)
            # print(type(WorkBook_County)) # Workbook
            WorkSheet_County_Race = WorkBook_County.getWorksheet("Race vax/unvax county")
            # print(type(WorkSheet_County_Race))
            df_full = WorkSheet_County_Race.data
            D_Full[county] = df_full
           
            print('Full', county)
        except:
            print('Full -- Fail', county)
    return D_Full




def get_Wisconsin_Raw(D_First, D_Full):
    
    #############
    L = []
    
    for county in D_First:
        try:
            df_first = D_First[county]
            df_full = D_Full[county]
            df_dict = process_county(county, df_first, df_full)
            L.append(df_dict)
        except:
            print('All -- Fail', county)
            
    #############


    Wisconsin = pd.DataFrame(L)
    
    return Wisconsin


In [11]:
D_First = get_Wisconsin_Raw_First()

First Wood County
First Winnebago County
First Waushara County
First Waupaca County
First Waukesha County
First Washington County
First Washburn County
First Walworth County
First Vilas County
First Vernon County
First Trempealeau County
First Taylor County
First St croix County
First Sheboygan County
First Shawano County
First Sawyer County
First Sauk County
First Rusk County
First Rock County
First Richland County
First Racine County
First Price County
First Portage County
First Polk County
First Pierce County
First Pepin County
First Ozaukee County
First Outagamie County
First Oneida County
First Oconto County
First Monroe County
First Milwaukee County
First Menominee County
First Marquette County
First Marinette County
First Marathon County
First Manitowoc County
First Lincoln County
First Langlade County
First Lafayette County
First La crosse County
First Kewaunee County
First Kenosha County
First Juneau County
First Jefferson County
First Jackson County
First Iron County
First Io

In [9]:
D_Full = get_Wisconsin_Raw_Full()

['County', 'SUM(Number With One Dose)', 'SUM(Number With Two Doses)', 'SUM(Popcount)', 'AGG(Number complete for map)', 'Latitude (generated)', 'Longitude (generated)', 'AGG(% complete)', 'AGG(Calc- Initiation or Full Coverage)', 'AGG(% at least 1 dose)']
Full Wood County
Full Winnebago County
Full Waushara County
Full Waupaca County
Full Waukesha County
Full Washington County
Full Washburn County
Full Walworth County
Full Vilas County
Full Vernon County
Full Trempealeau County
Full Taylor County
Full St croix County
Full Sheboygan County
Full Shawano County
Full Sawyer County
Full Sauk County
Full Rusk County
Full Rock County
Full Richland County
Full Racine County
Full Price County
Full Portage County
Full Polk County
Full Pierce County
Full Pepin County
Full Ozaukee County
Full Outagamie County
Full Oneida County
Full Oconto County
Full Monroe County
Full Milwaukee County
Full Menominee County
Full Marquette County
Full Marinette County
Full Marathon County
Full Manitowoc County
Full

In [13]:
import pandas as pd
Wisconsin = get_Wisconsin_Raw(D_First, D_Full)
# Wisconsin.to_csv('FinalResult/Raw_Wisconsin.csv')
print(Wisconsin.shape)
Wisconsin.head()

(72, 9)


,County,Numb_White,Numb_Black,Numb_Asian,Numb_American Indian,FullNumb_White,FullNumb_Black,FullNumb_Asian,FullNumb_American Indian
0,Wood,27534,139,603,136,24919,104,504,119
1,Winnebago,64146,697,2036,365,57174,513,1650,292
2,Waushara,6929,30,35,22,6357,22,34,19
3,Waupaca,17678,61,97,85,15677,41,80,73
4,Waukesha,179425,2402,7788,636,157184,1966,6394,515


In [18]:
state = 'Wisconsin'


def standardize_Wisconsin(Wisconsin, US_COUNTY_POPULATION):
    
    state = 'Wisconsin' ## 

    new_L = []

    for idx, row in Wisconsin.iterrows(): ## 
        d = row.to_dict()

        new_d = {}


        new_d['State'] = state
        new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
        

        RenameCounties = {
            'Fond Du Lac': 'Fond du Lac',
            'St Croix': 'St. Croix',
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']


        new_d['Numb_White'] = d['Numb_White']
        # new_d['Rate_White'] = d['Rate_White']

        new_d['Numb_Black'] = d['Numb_Black']
        # new_d['Rate_Black'] = d['Rate_Black']

        new_d['Numb_Asian'] = d['Numb_Asian']
        # new_d['Rate_Asian'] = d['Rate_Asian']

        new_d['Numb_AIndA'] = d['Numb_American Indian']
        # new_d['Rate_AIndA'] = d['Rate_American Indian']
        
        
        new_d['FullNumb_White'] = d['FullNumb_White']
        # new_d['Rate_White'] = d['Rate_White']

        new_d['FullNumb_Black'] = d['FullNumb_Black']
        # new_d['Rate_Black'] = d['Rate_Black']

        new_d['FullNumb_Asian'] = d['FullNumb_Asian']
        # new_d['Rate_Asian'] = d['Rate_Asian']

        new_d['FullNumb_AIndA'] = d['FullNumb_American Indian']

        new_L.append(new_d)

    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


        
    for Race in ['White', 'Black', 'Asian', 'AIndA']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate

        Rate = T['FullNumb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['FullRate_' + Race] = Rate



    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


    return Data, T


    
    
Data, T = standardize_Wisconsin(Wisconsin, US_COUNTY_POPULATION)

# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()


# diagnose_Rate(T)


(72, 28)


,State,County,Geographic Area Name,FullNumb_AIndA,FullNumb_Asian,FullNumb_Black,FullNumb_White,FullRate_AIndA,FullRate_Asian,FullRate_Black,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Wisconsin,Wood,"Wood, Wisconsin",119,504,104,24919,0.163912,0.348066,0.148148,...,0.398339,726,1036,1448,1660,702,1195,69122,70071,72999
1,Wisconsin,Winnebago,"Winnebago, Wisconsin",292,1650,513,57174,0.222222,0.313808,0.119247,...,0.406512,1314,2190,5258,6180,4302,5850,157796,160790,171907
2,Wisconsin,Waushara,"Waushara, Wisconsin",19,34,22,6357,0.098446,0.255639,0.042226,...,0.297842,193,350,133,186,521,644,23264,23568,24443
3,Wisconsin,Waupaca,"Waupaca, Wisconsin",73,80,41,15677,0.183417,0.267559,0.149091,...,0.357992,398,698,299,442,275,494,49381,49989,50990
4,Wisconsin,Waukesha,"Waukesha, Wisconsin",515,6394,1966,157184,0.379234,0.411560,0.277253,...,0.480041,1358,2854,15536,18090,7091,9611,373770,379839,404198


In [19]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Wisconsin']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,72.0,0.146425,0.068152,0.000000,0.101927,0.142803,0.176506,0.405806
1,FullRate_White,72.0,0.362710,0.064267,0.206017,0.330771,0.352553,0.389625,0.581851
2,Rate_Black,72.0,0.185352,0.085374,0.000000,0.124410,0.175192,0.226351,0.461290
3,Rate_White,72.0,0.403753,0.070411,0.236489,0.364686,0.398615,0.436757,0.610320
4,Rate_Parity,72.0,0.218401,0.083029,0.025597,0.161305,0.214570,0.264545,0.610320
5,FullRate_Parity,72.0,0.216285,0.073289,0.033199,0.167153,0.211912,0.255435,0.581851


## D. Ohio (Done)

https://coronavirus.ohio.gov/wps/portal/gov/covid-19/dashboards/covid-19-vaccine/covid-19-vaccination-dashboard

In [1]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
ts = TS()
ts.loads(url)

county = 'Williams'

WorkBook = ts.getWorkbook()


### 
WorkSheet_Map = WorkBook.getWorksheet("New Map")
SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}
WorkBook_County = WorkSheet_Map.select("county", county) # never use it


WorkBook_Race = WorkBook.setParameter('Key Metrics', "Race")

df_first = WorkBook_Race.getWorksheet('New Demographics').data
df_first

,Selected Demographic-value,Selected Demographic-alias,pop_pct_by_demographic-value,pop_pct_by_demographic-alias,SUM(Chosen Dose Status Metric)-value,SUM(Chosen Dose Status Metric)-alias
0,White,White,0.245245,0.245245,8691,8691
1,Unknown,Unknown,%null%,NA,313,313
2,Other,Other,%null%,NA,3130,3130
3,Black or African American,Black or African American,0.075,0.075,36,36
4,Asian,Asian,0.261905,0.261905,55,55
5,American Indian or Alaska Native,American Indian Alaska Native,0.083333,0.083333,11,11
6,Native Hawaiian or Other Pacific Islander,Native Hawaiian Pacific Islander,0.454545,0.454545,5,5
7,Multiracial,Multiracial,0.517815,0.517815,218,218


In [2]:
from tableauscraper import TableauScraper as TS

url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
ts = TS()
ts.loads(url)

county = 'Williams'

WorkBook = ts.getWorkbook()



### 
WorkSheet_Map = WorkBook.getWorksheet("New Map")
SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}
WorkBook_County = WorkSheet_Map.select("county", county) # never use it


WorkBook_Race = WorkBook.setParameter('Key Metrics', "Race")
WorkBook_Race = WorkBook.setParameter('View By', "Vaccine Completed**")


df_full = WorkBook_Race.getWorksheet('New Demographics').data
df_full

,Selected Demographic-value,Selected Demographic-alias,pop_pct_by_demographic-value,pop_pct_by_demographic-alias,SUM(Chosen Dose Status Metric)-value,SUM(Chosen Dose Status Metric)-alias
0,White,White,0.222501,0.222501,7885,7885
1,Unknown,Unknown,%null%,NA,263,263
2,Other,Other,%null%,NA,2955,2955
3,Black or African American,Black or African American,0.060417,0.060417,29,29
4,Asian,Asian,0.238095,0.238095,50,50
5,American Indian or Alaska Native,American Indian Alaska Native,0.05303,0.05303,7,7
6,Native Hawaiian or Other Pacific Islander,Native Hawaiian Pacific Islander,0.454545,0.454545,5,5
7,Multiracial,Multiracial,0.496437,0.496437,209,209


In [21]:
def process_county(county, df_first, df_full):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df_first.iterrows():
        d = row.to_dict()
        race = d['Selected Demographic-value']
        D['Numb_' + race] = d['SUM(Chosen Dose Status Metric)-value']
        D['Rate_' + race] = d['pop_pct_by_demographic-value']
        
    for idx, row in df_full.iterrows():
        d = row.to_dict()
        race = d['Selected Demographic-value']
        D['FullNumb_' + race] = d['SUM(Chosen Dose Status Metric)-value']
        D['FullRate_' + race] = d['pop_pct_by_demographic-value']
        
    return D


Data_County = process_county(county, df_first, df_full)
# Data_Full = process_county(county, df_full)

Data_County

{'County': 'Williams',
 'Numb_White': 8691,
 'Rate_White': 0.245245216998702,
 'Numb_Unknown': 313,
 'Rate_Unknown': '%null%',
 'Numb_Other': 3130,
 'Rate_Other': '%null%',
 'Numb_Black or African American': 36,
 'Rate_Black or African American': 0.075,
 'Numb_Asian': 55,
 'Rate_Asian': 0.2619047619047619,
 'Numb_American Indian or Alaska Native': 11,
 'Rate_American Indian or Alaska Native': 0.0833333333333333,
 'Numb_Native Hawaiian or Other Pacific Islander': 5,
 'Rate_Native Hawaiian or Other Pacific Islander': 0.4545454545454546,
 'Numb_Multiracial': 218,
 'Rate_Multiracial': 0.5178147268408552,
 'FullNumb_White': 7885,
 'FullRate_White': 0.2225012698233535,
 'FullNumb_Unknown': 263,
 'FullRate_Unknown': '%null%',
 'FullNumb_Other': 2955,
 'FullRate_Other': '%null%',
 'FullNumb_Black or African American': 29,
 'FullRate_Black or African American': 0.0604166666666667,
 'FullNumb_Asian': 50,
 'FullRate_Asian': 0.2380952380952381,
 'FullNumb_American Indian or Alaska Native': 7,
 'Fu

In [6]:

from tableauscraper import TableauScraper as TS


def get_Ohio_Raw_First():
    
        
    url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"

    
    #############
    ts = TS()
    ts.loads(url)

    WorkBook = ts.getWorkbook()

    WorkSheet_Map = WorkBook.getWorksheet("New Map")
    SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}

    
    
    D_First = {}
    for county in SelectionDict['county']:
        try:
            ts = TS()
            ts.loads(url)

            WorkBook = ts.getWorkbook()

            WorkSheet_Map = WorkBook.getWorksheet("New Map")
            WorkBook_County = WorkSheet_Map.select("county", county) # never use it
            
            
            WorkBook_Race = WorkBook.setParameter('Key Metrics', "Race")
            df_first = WorkBook_Race.getWorksheet('New Demographics').data
            D_First[county] = df_first
            print('First', county)
            # df_first
        except:
            print('First -- Fail', county)
    #############
    
    return D_First

D_First = get_Ohio_Raw_First()



First Wyandot
First Wood
First Williams
First Wayne
First Washington
First Warren
First Vinton
First Van Wert
First Union
First Tuscarawas
First Trumbull
First Summit
First Stark
First Shelby
First Seneca
First Scioto
First Sandusky
First Ross
First Richland
First Putnam
First Preble
First Portage
First Pike
First Pickaway
First Perry
First Paulding
First Ottawa
First Noble
First Muskingum
First Morrow
First Morgan
First Montgomery
First Monroe
First Miami
First Mercer
First Meigs
First Medina
First Marion
First Mahoning
First Madison
First Lucas
First Lorain
First Logan
First Licking
First Lawrence
First Lake
First Knox
First Jefferson
First Jackson
First Huron
First Holmes
First Hocking
First Highland
First Henry
First Harrison
First Hardin
First Hancock
First Hamilton
First Guernsey
First Greene
First Geauga
First Gallia
First Fulton
First Franklin
First Fayette
First Fairfield
First Erie
First Delaware
First Defiance
First Darke
First Cuyahoga
First Crawford
First Coshocton
First C

In [7]:

from tableauscraper import TableauScraper as TS



def get_Ohio_Raw_Full():
    url = "https://public.tableau.com/views/VaccineAdministrationMetricsDashboard/PublicCountyDash"
    ts = TS()
    ts.loads(url)

    # county = 'Williams'

    WorkBook = ts.getWorkbook()

    WorkSheet_Map = WorkBook.getWorksheet("New Map")
    SelectionDict = {i['column']:i['values'] for i in WorkSheet_Map.getSelectableItems()}


   
    D_Full = {}
    for county in SelectionDict['county']:
            
        try:
            ts = TS()
            ts.loads(url)

            WorkBook = ts.getWorkbook()

            WorkSheet_Map = WorkBook.getWorksheet("New Map")
            WorkBook_County = WorkSheet_Map.select("county", county) # never use it

            WorkBook_Race = WorkBook.setParameter('Key Metrics', "Race")
            WorkBook_Race = WorkBook.setParameter('View By', "Vaccine Completed**")

            df_full = WorkBook_Race.getWorksheet('New Demographics').data
            
            D_Full[county] = df_full
           
            print('Full', county)
        except:
            print('Full -- Fail', county)
    return D_Full



D_Full = get_Ohio_Raw_Full()



Full Wyandot
Full Wood
Full Williams
Full Wayne
Full Washington
Full Warren
Full Vinton
Full Van Wert
Full Union
Full Tuscarawas
Full Trumbull
Full Summit
Full Stark
Full Shelby
Full Seneca
Full Scioto
Full Sandusky
Full Ross
Full Richland
Full Putnam
Full Preble
Full Portage
Full Pike
Full Pickaway
Full Perry
Full Paulding
Full Ottawa
Full Noble
Full Muskingum
Full Morrow
Full Morgan
Full Montgomery
Full Monroe
Full Miami
Full Mercer
Full Meigs
Full Medina
Full Marion
Full Mahoning
Full Madison
Full Lucas
Full Lorain
Full Logan
Full Licking
Full Lawrence
Full Lake
Full Knox
Full Jefferson
Full Jackson
Full Huron
Full Holmes
Full Hocking
Full Highland
Full Henry
Full Harrison
Full Hardin
Full Hancock
Full Hamilton
Full Guernsey
Full Greene
Full Geauga
Full Gallia
Full Fulton
Full Franklin
Full Fayette
Full Fairfield
Full Erie
Full Delaware
Full Defiance
Full Darke
Full Cuyahoga
Full Crawford
Full Coshocton
Full Columbiana
Full Clinton
Full Clermont
Full Clark
Full Champaign
Full Carrol

In [22]:



def get_Ohio_Raw(D_First, D_Full):
    
    #############
    L = []
    
    for county in D_First:
        try:
            df_first = D_First[county]
            df_full = D_Full[county]
            df_dict = process_county(county, df_first, df_full)
            L.append(df_dict)
        except:
            print('All -- Fail', county)
            
    #############


    Ohio = pd.DataFrame(L)
    
    return Ohio



Ohio = get_Ohio_Raw(D_First, D_Full)

Ohio.head()

,County,Numb_White,Rate_White,Numb_Unknown,Rate_Unknown,Numb_Other,Rate_Other,Numb_Black or African American,Rate_Black or African American,Numb_Asian,...,FullNumb_Black or African American,FullRate_Black or African American,FullNumb_Asian,FullRate_Asian,FullNumb_American Indian or Alaska Native,FullRate_American Indian or Alaska Native,FullNumb_Native Hawaiian or Other Pacific Islander,FullRate_Native Hawaiian or Other Pacific Islander,FullNumb_Multiracial,FullRate_Multiracial
0,Wyandot,6742,0.317794,866,%null%,386,%null%,18,0.206897,45,...,15,0.172414,40,0.261438,8.0,0.123077,5.0,0.625,61,0.250000
1,Wood,54476,0.447423,2060,%null%,3785,%null%,1088,0.293737,1545,...,913,0.246490,1359,0.562267,128.0,0.299065,43.0,0.741379,337,0.137271
2,Williams,8691,0.245245,313,%null%,3130,%null%,36,0.075000,55,...,29,0.060417,50,0.238095,7.0,0.053030,5.0,0.454545,209,0.496437
3,Wayne,34701,0.314957,1585,%null%,1483,%null%,559,0.295143,590,...,463,0.244456,472,0.374603,51.0,0.209016,16.0,0.32,166,0.079616
4,Washington,20026,0.348496,1784,%null%,609,%null%,198,0.252874,132,...,166,0.212005,103,0.250608,30.0,0.177515,7.0,0.5,54,0.050467


In [23]:
Ohio.columns

Index(['County', 'Numb_White', 'Rate_White', 'Numb_Unknown', 'Rate_Unknown',
       'Numb_Other', 'Rate_Other', 'Numb_Black or African American',
       'Rate_Black or African American', 'Numb_Asian', 'Rate_Asian',
       'Numb_American Indian or Alaska Native',
       'Rate_American Indian or Alaska Native',
       'Numb_Native Hawaiian or Other Pacific Islander',
       'Rate_Native Hawaiian or Other Pacific Islander', 'Numb_Multiracial',
       'Rate_Multiracial', 'FullNumb_White', 'FullRate_White',
       'FullNumb_Unknown', 'FullRate_Unknown', 'FullNumb_Other',
       'FullRate_Other', 'FullNumb_Black or African American',
       'FullRate_Black or African American', 'FullNumb_Asian',
       'FullRate_Asian', 'FullNumb_American Indian or Alaska Native',
       'FullRate_American Indian or Alaska Native',
       'FullNumb_Native Hawaiian or Other Pacific Islander',
       'FullRate_Native Hawaiian or Other Pacific Islander',
       'FullNumb_Multiracial', 'FullRate_Multiracial'],
 

In [24]:

# print(len(L))

import numpy as np

state = 'Ohio' ## 

new_L = []

for idx, row in Ohio.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    RenameCounties = {
        # 'Mckean': 'McKean'
        'Mccormick': 'McCormick'
    }
    if new_d['County'] in RenameCounties:
        new_d['County'] = RenameCounties[new_d['County']]

    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']

    new_d['Numb_White'] = d['Numb_White']
    
    new_d['Numb_Black'] = d['Numb_Black or African American']
    
    new_d['Numb_Asian'] = d['Numb_Asian']
    
    new_d['Numb_AIndA'] = d['Numb_American Indian or Alaska Native']
    
    
    
    new_d['FullNumb_White'] = d['FullNumb_White']
    
    new_d['FullNumb_Black'] = d['FullNumb_Black or African American']
    
    new_d['FullNumb_Asian'] = d['FullNumb_Asian']
    
    new_d['FullNumb_AIndA'] = d['FullNumb_American Indian or Alaska Native']
    
    
    
    new_L.append(new_d)

    
    
Data = pd.DataFrame(new_L)

not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian', 'AIndA']:

    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['FullNumb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['FullRate_' + Race] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]


print(T.shape)
T.head()

(88, 28)


,State,County,Geographic Area Name,FullNumb_AIndA,FullNumb_Asian,FullNumb_Black,FullNumb_White,FullRate_AIndA,FullRate_Asian,FullRate_Black,...,Rate_White,Total_AIndA,Total_AIndAC,Total_Asian,Total_AsianC,Total_Black,Total_BlackC,Total_White,Total_WhiteC,Total_Whole
0,Ohio,Wyandot,"Wyandot, Ohio",8.0,40,15,6156,0.123077,0.261438,0.172414,...,0.317794,65,158,153,190,87,210,21215,21452,21772
1,Ohio,Wood,"Wood, Ohio",128.0,1359,913,49629,0.299065,0.562267,0.246490,...,0.447423,428,1187,2417,3082,3704,4931,121755,124089,130817
2,Ohio,Williams,"Williams, Ohio",7.0,50,29,7885,0.053030,0.238095,0.060417,...,0.245245,132,316,210,280,480,678,35438,35837,36692
3,Ohio,Wayne,"Wayne, Ohio",51.0,472,463,30088,0.209016,0.374603,0.244456,...,0.314957,244,871,1260,1660,1894,3082,110177,112174,115710
4,Ohio,Washington,"Washington, Ohio",30.0,103,166,18243,0.177515,0.250608,0.212005,...,0.348496,169,655,411,618,783,1278,57464,58470,59911


In [25]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
T['FullRate_Parity']  = T['FullRate_White'] - T['FullRate_Black']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

['Ohio']


,name,count,mean,std,min,25%,50%,75%,max
0,FullRate_Black,88.0,0.184523,0.060602,0.045977,0.143089,0.184640,0.222256,0.391799
1,FullRate_White,88.0,0.300442,0.068639,0.110306,0.258627,0.283055,0.341583,0.478406
2,Rate_Black,88.0,0.220715,0.070532,0.075000,0.173538,0.220470,0.262925,0.467564
3,Rate_White,88.0,0.335374,0.078459,0.122929,0.280994,0.314619,0.384361,0.533213
4,Rate_Parity,88.0,0.114659,0.060280,-0.133756,0.082648,0.110988,0.153478,0.253268
5,FullRate_Parity,88.0,0.115919,0.054671,-0.082207,0.086516,0.113811,0.151015,0.241482


In [26]:
T.to_csv(state + '.csv')

## D. South Carolina* (Done)

In [24]:
from tableauscraper import TableauScraper as TS

url = 'https://public.tableau.com/views/COVIDVaccineDashboard/RECIPIENTVIEW'
ts = TS()
ts.loads(url)
ws = ts.getWorksheet("Vaccine Map By SC residents PEOPLE")
ws.data

,Recipient County for maps-value,Recipient County for maps-alias,Recipient Zip Code for maps-alias,Latitude (generated)-value,Latitude (generated)-alias,Longitude (generated)-value,Longitude (generated)-alias,AGG(Count individuals with Suppression )-alias,AGG(Count individuals with Suppression for bar graph)-alias
0,York,York,29945,32.6803,32.680,-80.7941,-80.794,"1,509","1,509"
1,York,York,29944,32.7909,32.791,-81.0332,-81.033,"1,660","1,660"
2,York,York,29943,32.4596,32.460,-81.1515,-81.152,135,135
3,York,York,29941,32.5534,32.553,-80.8144,-80.814,387,387
4,York,York,29940,32.5557,32.556,-80.6994,-80.699,"1,167","1,167"
...,...,...,...,...,...,...,...,...,...
592,Abbeville,Abbeville,0,0,0,0,0,0,0
593,Abbeville,Abbeville,0,0,0,0,0,0,0
594,Abbeville,Abbeville,0,0,0,0,0,0,0
595,Abbeville,Abbeville,0,0,0,0,0,0,0


In [25]:
import time 

def process_county(county, df):
    df = df.replace(-1, 0)
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for k, racedf in df.groupby('Assigned Race-value'):
        Numb = racedf['AGG(Count individuals with Suppression )-alias'].sum()
        if 'Asian' in k:
            k = 'Asian'
        # print(k)
        # print(Numb)å
        D['Numb_' + k] = Numb
    return D


In [ ]:
county = 'Williamsburg'

wb = ws.setFilter('Recipient County for maps', county)
countyWs = wb.getWorksheet("Final Age xSex x Race REC")

df = countyWs.data

# southcarolina.process_county(county, df)

process_county(county, df)

In [ ]:
county = 'York'

wb = ws.setFilter('Recipient County for maps', county)
countyWs = wb.getWorksheet("Final Age xSex x Race REC")

df = countyWs.data

# southcarolina.process_county(county, df)
process_county(county, df)

In [ ]:


def get_SouthCarolina_Raw():
    url = 'https://public.tableau.com/views/COVIDVaccineDashboard/RECIPIENTVIEW'
    ts = TS()
    ts.loads(url)
    ws = ts.getWorksheet("Vaccine Map By SC residents PEOPLE")

    # show filter columns and values
    filters = ws.getFilters()
    counties = next(iter([
        t["values"]
        for t in filters
        if t["column"] == "Recipient County for maps"
    ]))

    L = []
    for county in counties:
        
        wb = ws.setFilter('Recipient County for maps', county)
        countyWs = wb.getWorksheet("Final Age xSex x Race REC")

        # print(countyWs.data)
        df = countyWs.data
        d = process_county(county, df)
        L.append(d)
        print('Success\t\t', county)
        time.sleep(0.5)
        
    SouthCarolina = pd.DataFrame(L)
    return SouthCarolina



# SouthCarolina = southcarolina.get_SouthCarolina_Raw()
SouthCarolina = get_SouthCarolina_Raw()
SouthCarolina = SouthCarolina.replace(0, np.nan)
print(SouthCarolina.shape)
SouthCarolina.head()

In [ ]:

def standardize_SouthCarolina(SouthCarolina, US_COUNTY_POPULATION):
    
    state = 'South Carolina' ## 

    new_L = []

    for idx, row in SouthCarolina.iterrows(): ## 
        d = row.to_dict()

        new_d = {}
        new_d['State'] = state
        new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])

        RenameCounties = {
            'Mckean': 'McKean',
            'Mccormick': 'McCormick'
        }
        if new_d['County'] in RenameCounties:
            new_d['County'] = RenameCounties[new_d['County']]

        new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
        
        new_d['Numb_White'] = d['Numb_White']
        new_d['Rate_White'] = np.nan

        new_d['Numb_Black'] = d['Numb_Black']
        new_d['Rate_Black'] = np.nan

        new_d['Numb_Asian'] = d['Numb_Asian']
        new_d['Rate_Asian'] = np.nan

        new_d['Numb_AIndA'] = np.nan
        new_d['Rate_AIndA'] = np.nan

        new_d['Numb_Other'] = d['Numb_Other']
        new_d['Rate_Other'] = np.nan

        new_d['Date'] = np.nan

        new_L.append(new_d)


    Data = pd.DataFrame(new_L)

    not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
    if len(not_included) > 0:
        print('Not included Counties', not_included)


    T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


    for Race in ['White', 'Black', 'Asian']:
        Rate = T['Numb_' + Race] / T['Total_' + Race]
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race] = Rate

        Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
        Rate = Rate.replace([-np.inf, np.inf], np.nan)
        T['Rate_' + Race + 'C'] = Rate


    T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
    # print(T.shape)
    # T.head()
    return Data, T


In [ ]:
state = 'South Carolina'
Data, T = standardize_SouthCarolina(SouthCarolina, US_COUNTY_POPULATION)
# T.to_csv('FinalResult/T_' + state +'.csv', index = False)
print(T.shape)
T.head()

In [ ]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
# STATE_VACCINE_TMP = {}
# STATE_VACCINE_TMP[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

## D. Oregon* (Done)

In [ ]:
from tableauscraper import TableauScraper as TS

url = 'https://public.tableau.com/views/OregonCOVID-19VaccinationTrends/OregonCountyVaccinationTrends'

ts = TS()
ts.loads(url)

In [ ]:
worksheet = ts.getWorksheet("County Map Per Capita new")

SelectionDict = {i['column']:i['values'] for i in worksheet.getSelectableItems()}

counties = SelectionDict['Recip Address County']


def process_county(county, df):
    county = county.replace(' County', '')
    D = {}
    D['County'] = county
    for idx, row in df.iterrows():
        d = row.to_dict()
        race = d['Demographic Category-value']
        data = d['AGG(People Count Display)-alias']
        data = float(data.replace(',','')) if data!= 'Suppressed' else np.nan
        D['Numb_' + race] = data
    return D

county = 'COOS'

dashboard = worksheet.select('Recip Address County', county)
df = dashboard.getWorksheet('County - Race-Ethnicity').data
d = process_county(county, df)
d

In [ ]:
L = []


url = 'https://public.tableau.com/views/OregonCOVID-19VaccinationTrends/OregonCountyVaccinationTrends'

ts = TS()
ts.loads(url)

worksheet = ts.getWorksheet("County Map Per Capita new")


        
for county in counties:
    dashboard = worksheet.select('Recip Address County', county)
    df = dashboard.getWorksheet('County - Race-Ethnicity').data
    df_dict = process_county(county, df)
    L.append(df_dict)
    print('Success\t\t', county)

Oregon = pd.DataFrame(L)
print(Oregon.shape)
Oregon.head()

In [ ]:
# print(len(L))

import numpy as np

state = 'Oregon' ## 
new_L = []


for idx, row in Oregon.iterrows(): ## 
    d = row.to_dict()
    
    new_d = {}
    
    new_d['State'] = state
    new_d['County'] = ' '.join([i.lower().capitalize() for i in d['County'].split(' ')])
    new_d['Geographic Area Name'] = new_d['County'] + ', ' + new_d['State']
    
    new_d['Numb_White'] = d['Numb_White'] if d['Numb_White']!= '-' else np.nan
    new_d['Rate_White'] = np.nan
    
    new_d['Numb_Black'] = d['Numb_Black'] if d['Numb_Black']!= '-' else np.nan
    new_d['Rate_Black'] = np.nan
    
    new_d['Numb_Asian'] = d['Numb_Asian'] if d['Numb_Asian']!= '-' else np.nan
    new_d['Rate_Asian'] = np.nan
    
    new_d['Date'] = np.nan
    
    new_L.append(new_d)
    
    
Data = pd.DataFrame(new_L)
not_included = list(set(Data['Geographic Area Name'].unique()) - set(US_COUNTY_POPULATION['Geographic Area Name'].unique()))
if len(not_included) > 0:
    print('Not included Counties', not_included)


T = pd.merge(Data, US_COUNTY_POPULATION[US_COUNTY_POPULATION.columns[3:]], on = 'Geographic Area Name')


for Race in ['White', 'Black', 'Asian']:
    Rate = T['Numb_' + Race] / T['Total_' + Race]
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race] = Rate

    Rate = T['Numb_' + Race] / T['Total_' + Race + 'C']
    Rate = Rate.replace([-np.inf, np.inf], np.nan)
    T['Rate_' + Race + 'C'] = Rate


T = T[list(T.columns[:3]) + list(T.columns[3:].sort_values())]
print(T.shape)
T.head()

In [ ]:
T['Rate_Parity']  = T['Rate_White'] - T['Rate_Black']
# T['Rate_ParityC'] = T['Rate_WhiteC'] - T['Rate_BlackC']
STATE_VACCINE[state] = T
print([i for i in STATE_VACCINE])
diagnose_Rate(T)

# All in One

## Read Previous Vaccine Data

In [ ]:
# import os
# path = ''

# print('Input Historical STATE_VACCINE')
# print('For example: RaceVaccineBackup/STATE_VACCINE_2021-03-23.p')

# L = []
# while True:
    
#     path = input()
    
#     if os.path.isfile(path):
#         print('Path Exists!')
#         L.append(path)
#     elif path != '':
#         print('Wrong Path')
#     else:
#         break

# print('End')

In [ ]:
# import pickle

# Historical_Path = {i.split('_')[-1].replace('.p', ''): i for i in L }

# Historical_Data = {}
# for date, path in Historical_Path.items():
#     with open(path, 'rb' ) as f:
#         Historical_Data[date] = pickle.load(f)
    
    
# [i for i in Historical_Data]

## Show Parity for Each State

In [ ]:
# ######################################

# # Today_Date = str(datetime.now().date())
# print(Today_Date)
# ######################################


# for state in STATE_VACCINE:
    
#     print(state)
#     cols = ['name',  'date', 'count', 'mean', 'min','50%', 'max']
    
    
    
#     for date, Old_STATE_VACCINE in Historical_Data.items():
#         x = diagnose_Rate(Old_STATE_VACCINE[state], state = state, date = date)
#         print(x[cols])
    
#     x = diagnose_Rate(STATE_VACCINE[state], state = state, date = Today_Date)
    
#     print(x[cols])
#     print('\n')
    
    
# # pd.concat(L).to_csv('compare.csv')

##  Show Some Specials Cases

In [ ]:
# # state = 'Texas'
# DF = pd.concat([v for k,v in STATE_VACCINE.items()]) 

# data_cols = ['State', 'County', 'Geographic Area Name',
#              'Total_Black', 'Total_White',
#              'Numb_Black', 'Numb_White','Rate_Black','Rate_White', 'Total_Whole', 'Rate_Parity']

# DF[DF['Rate_Black'] > 0.3][data_cols]

In [28]:
STATE_VACCINE = {}



for i in ['New York', 'Ohio', 'Oregon', 'South Carolina', 'Wisconsin']:
    STATE_VACCINE[i] = pd.read_csv(i +'.csv')
    

In [31]:
# STATE_VACCINE['Ohio']

In [29]:
[i for i in STATE_VACCINE]

['New York', 'Ohio', 'Oregon', 'South Carolina', 'Wisconsin']

## Save Data

In [33]:

import pickle 
Today_Date = '2021-05-20'
path = 'CountyVaccine/Workplace/STATE_VACCINE_' + Today_Date + '_Tableau.p'

while True:
    print('The data will be saved here, do you agree?' )
    print(path)
    print('If yes, please enter: Yes')
    x = input()
    if x =='Yes':
        break


with open(path, 'wb') as f:
    pickle.dump(STATE_VACCINE, f)
    
    
print('Successfully Saved!')

The data will be saved here, do you agree?
CountyVaccine/Workplace/STATE_VACCINE_2021-05-20_Tableau.p
If yes, please enter: Yes
Yes
Successfully Saved!
